# MNE-Python Basic Preprocess
https://mne.tools/stable/documentation/index.html

In [138]:
import pyxdf
import mne
import numpy as np
import pandas as pd
streams, header = pyxdf.load_xdf("C:/Users/Toey/Desktop/data_ssvep/Toey/exp1/block_2.xdf")
# streams, header = pyxdf.load_xdf("Pipo_1_5_test1.xdf") #Example Data from Lab Recoder
# streams, header = pyxdf.load_xdf("C:/Users/Toey/Desktop/SSVEP_data/sub-P001/ses-S001/eeg/sub-P001_ses-S001_task-Default_run-001_eeg.xdf") #Example Data from Lab Recoder

In [139]:
streams

[{'info': defaultdict(list,
              {'name': ['obci_eeg1'],
               'type': ['EEG'],
               'channel_count': ['8'],
               'nominal_srate': ['250'],
               'channel_format': ['float32'],
               'source_id': ['openbcigui'],
               'version': ['1.1000000000000001'],
               'created_at': ['97866.327942699994'],
               'uid': ['2e5886a9-33e7-4ade-be69-41ca8fd3a01e'],
               'session_id': ['default'],
               'hostname': ['DESKTOP-5KJIQRK'],
               'v4address': [None],
               'v4data_port': ['16572'],
               'v4service_port': ['16572'],
               'v6address': [None],
               'v6data_port': ['16572'],
               'v6service_port': ['16572'],
               'desc': [None],
               'stream_id': 1,
               'effective_srate': 250.3560058307626}),
  'footer': {'info': defaultdict(list,
               {'first_timestamp': ['101410.7625467'],
                'last_

In [140]:
raw_data = streams[0]["time_series"].T #From Steam variable this query is EEG data

In [141]:
raw_data.shape # It contain with 8 channel of EGG data

(8, 229960)

In [142]:
channels =['Fz','Cz','P4','Pz','P3','P08','Oz','P07'] #Set your target EEG channel name
info = mne.create_info(
    ch_names= channels,
    ch_types= ['eeg']*len(channels),
    sfreq= 250 #OpenBCI Frequency acquistion
)
# Create MNE rawarray
raw_mne = mne.io.RawArray(raw_data, info, verbose=False)

In [143]:
raw_mne

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,8 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,0.00 Hz
Lowpass,125.00 Hz


# Create MNE Epoch

In [144]:
event_index = streams[1]["time_series"].T[0] #Get all event marker of experiment 
event_timestamp = streams[1]["time_stamps"].T #Timestamp when event marked

In [145]:
event_index

array([1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0,
       3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0,
       0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2,
       0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0, 4,
       0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0,
       2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0,
       4, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1,
       0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3,
       0, 4, 0, 4])

In [146]:
c = 0
for i in range(len(event_index)):
    if event_index[i] == 0:
        c += 1
print(f'number of {c}')

number of 99


In [172]:
len(event_timestamp)

180

In [148]:
events = np.column_stack((np.array(event_timestamp, dtype = int),
                        np.zeros(len(event_timestamp), dtype = int),
                        np.array(event_index, dtype = int)))
fre1 = 8
fre2 = 10
fre3 = 12
fre4 = 14
events_id = { # Set up your event name
     f'natural': 0,
     f'Frequency {fre1} Hz': 1,
     f'Frequency {fre2} Hz': 2,
     f'Frequency {fre3} Hz': 3,
     f'Frequency {fre4} Hz': 4,
}

mne_epochs = mne.Epochs(raw_mne, events, 
        tmin= -5,     # init timestamp of epoch (0 means trigger timestamp same as event start)
        tmax=10,    # final timestamp (10 means set epoch duration 10 second)
        event_id =events_id,
        preload = True,
        event_repeated='drop',
        baseline=(-0.5, 0)
    )
# mne_epochs = mne.Epochs(raw_mne, events, 
#         tmin= -5,     # init timestamp of epoch (0 means trigger timestamp same as event start)
#         tmax= 20,    # final timestamp (10 means set epoch duration 10 second)
#         event_id =events_id,
#         preload = True,
#         event_repeated='drop',
#         baseline=(-0.5, 0)
#     )

Not setting metadata
180 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 180 events and 3751 original time points ...
0 bad epochs dropped


In [149]:
mne_epochs

Number of events,180
Events,Frequency 10 Hz: 20Frequency 12 Hz: 20Frequency 14 Hz: 21Frequency 8 Hz: 20natural: 99
Time range,-5.000 – 10.000 s
Baseline,-0.500 – 0.000 s


In [150]:
# mne_epochs.plot(scalings = 200)
# mne_epochs.filter(l_freq=8.0, h_freq=13.0, method = 'iir', iir_params= {"order": 6, "ftype":'butter'}).plot(scalings = 100)
# print('...')

In [151]:
print(mne_epochs.get_data().shape)
mne_epochs.get_data()

(180, 8, 3751)


C:\Users\Toey\AppData\Local\Temp\ipykernel_23128\23489598.py:1: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(mne_epochs.get_data().shape)
C:\Users\Toey\AppData\Local\Temp\ipykernel_23128\23489598.py:2: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  mne_epochs.get_data()


array([[[-6.64405056e+02, -6.93347439e+01,  6.07945530e+02, ...,
         -4.02285916e+02, -7.00681424e+02, -8.06892361e+01],
        [ 3.41223803e+02,  1.52150561e+02, -1.33281079e+02, ...,
          2.59298580e+01,  1.13951342e+02, -3.76385014e+01],
        [-7.02811111e+03, -6.59086111e+03,  3.01513889e+03, ...,
          1.06938889e+03, -6.96011111e+03, -4.68611111e+03],
        ...,
        [-7.45796015e+02, -9.18956241e+01,  6.87911993e+02, ...,
         -3.41810663e+02, -7.05942987e+02, -2.82159366e+01],
        [-8.30711908e+02, -9.27016543e+01,  7.35676031e+02, ...,
         -3.07233881e+02, -6.73892084e+02,  6.89013730e+01],
        [-8.08229016e+02, -9.03803827e+01,  7.15579090e+02, ...,
         -2.97804211e+02, -6.57711926e+02,  6.39584845e+01]],

       [[-6.54081024e+02, -6.67441096e+01,  6.12145539e+02, ...,
         -3.77076141e+02, -6.81014617e+02, -6.59618831e+01],
        [ 3.46032025e+02,  1.68245892e+02, -1.22639850e+02, ...,
          2.68494079e+01,  1.26537884e

In [152]:
dataset = mne_epochs.to_data_frame()
id_df = pd.DataFrame({"id": range(len(dataset))})
dataset = pd.concat([id_df,dataset], axis=1)

In [153]:
labels = mne_epochs.events[:,-1]
labels

array([1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0,
       3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0,
       0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2,
       0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0, 4,
       0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0,
       2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0,
       4, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1,
       0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 0, 1, 0, 2, 0, 3,
       0, 4, 0, 4])

In [154]:
dataset.head(5)

,id,time,condition,epoch,Fz,Cz,P4,Pz,P3,P08,Oz,P07
0,0,-5.000,Frequency 8 Hz,0,-6.644051e+08,3.412238e+08,-7.028111e+09,-8.324441e+08,3.788887e+09,-7.457960e+08,-8.307119e+08,-8.082290e+08
1,1,-4.996,Frequency 8 Hz,0,-6.933474e+07,1.521506e+08,-6.590861e+09,-1.198258e+08,8.993210e+08,-9.189562e+07,-9.270165e+07,-9.038038e+07
2,2,-4.992,Frequency 8 Hz,0,6.079455e+08,-1.332811e+08,3.015139e+09,6.935095e+08,-2.930002e+09,6.879120e+08,7.356760e+08,7.155791e+08
3,3,-4.988,Frequency 8 Hz,0,4.591057e+08,-1.257489e+08,7.605389e+09,5.220940e+08,-2.466070e+09,5.432064e+08,5.516987e+08,5.384414e+08
4,4,-4.984,Frequency 8 Hz,0,-1.531980e+08,1.238527e+08,2.708889e+09,-2.141501e+08,8.791149e+08,-1.544806e+08,-2.062710e+08,-1.936675e+08


In [155]:
dataset.describe()

,id,time,epoch,Fz,Cz,P4,Pz,P3,P08,Oz,P07
count,675180.000000,675180.000000,675180.000000,6.751800e+05,6.751800e+05,6.751800e+05,6.751800e+05,6.751800e+05,6.751800e+05,6.751800e+05,6.751800e+05
mean,337589.500000,2.500000,89.500000,-3.924711e+07,-2.086566e+07,-4.583991e+07,3.542889e+06,-2.540769e+06,7.624056e+06,1.205506e+07,1.222687e+07
std,194907.821713,4.331285,51.960761,3.987438e+08,1.313281e+08,5.206553e+09,4.718538e+08,1.998911e+09,4.504716e+08,4.830583e+08,4.697073e+08
min,0.000000,-5.000000,0.000000,-8.275278e+08,-3.399382e+08,-8.843847e+09,-8.679579e+08,-3.539536e+09,-8.189088e+08,-8.808743e+08,-8.405842e+08
25%,168794.750000,-1.252000,44.750000,-4.234493e+08,-1.318640e+08,-5.203475e+09,-4.573508e+08,-1.925979e+09,-4.330676e+08,-4.584490e+08,-4.462331e+08
50%,337589.500000,2.500000,89.500000,-2.371617e+06,-2.376495e+07,3.950645e+08,4.881675e+07,-1.657719e+08,4.889050e+07,5.419428e+07,5.310199e+07
75%,506384.250000,6.252000,134.250000,3.394208e+08,8.937379e+07,5.135957e+09,4.557293e+08,1.943868e+09,4.377861e+08,4.763646e+08,4.618256e+08
max,675179.000000,10.000000,179.000000,6.573542e+08,3.504328e+08,7.925246e+09,8.079749e+08,3.880053e+09,7.917300e+08,8.545507e+08,8.330107e+08


In [156]:
dataset.groupby('condition').size()

condition
Frequency 10 Hz     75020
Frequency 12 Hz     75020
Frequency 14 Hz     78771
Frequency 8 Hz      75020
natural            371349
dtype: int64

In [157]:
feature_columns = ['Fz','Cz','P4','Pz','P3','P08','Oz','P07']
X = dataset[feature_columns].values
y = dataset['condition'].values

In [158]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [159]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
print(y_test)

[2 4 4 ... 4 4 2]


In [160]:
import matplotlib.pyplot as plt
import seaborn as sns

In [161]:
# from pandas.plotting import parallel_coordinates
# plt.figure(figsize=(15,10))
# parallel_coordinates(dataset.drop("id", axis=1), "condition")
# plt.title('Parallel Coordinates Plot', fontsize=20, fontweight='bold')
# # plt.xlabel('Features', fontsize=15)
# plt.ylabel('Features values', fontsize=15)
# plt.legend(loc=1, prop={'size': 15}, frameon=True,shadow=True, facecolor="white", edgecolor="black")
# plt.show()

In [162]:
# from pandas.plotting import andrews_curves
# plt.figure(figsize=(15,10))
# andrews_curves(dataset.drop("id", axis=1), "condition")
# plt.title('Andrews Curves Plot', fontsize=20, fontweight='bold')
# plt.legend(loc=1, prop={'size': 15}, frameon=True,shadow=True, facecolor="white", edgecolor="black")
# plt.show()

In [163]:
# plt.figure()
# sns.pairplot(dataset.drop("id", axis=1), hue = "condition", size=3, markers=["o", "s", "D"])
# plt.show()

In [164]:
# plt.figure()
# dataset.drop("id", axis=1).boxplot(by="condition", figsize=(15, 10))
# plt.show()

In [165]:
# Fitting clasifier to the Training set
# Loading libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

# Instantiate learning model (k = 3)
classifier = KNeighborsClassifier(n_neighbors=3)

# Fitting the model
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)
print(y_pred)

[0 2 4 ... 0 4 4]


In [166]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[  294  2434   436     4 11950]
 [ 3207    19   246     0 11542]
 [ 1789   287    16     1 13520]
 [ 1206    98    28     0 13626]
 [24129 13842  8245     4 28113]]


In [171]:
accuracy = accuracy_score(y_test, y_pred)*100
print(y_test)
print(y_pred)
print('Accuracy of our model is equal ' + str(round(accuracy, 2)) + ' %.')

[2 4 4 ... 4 4 2]
[0 2 4 ... 0 4 4]
Accuracy of our model is equal 21.06 %.


In [168]:
# # creating list of K for KNN
# k_list = list(range(1,50,2))
# # creating list of cv scores
# cv_scores = []

# # perform 10-fold cross validation
# for k in k_list:
#     knn = KNeighborsClassifier(n_neighbors=k)
#     scores = cross_val_score(knn, X_train, y_train, cv=10, scoring='accuracy')
#     cv_scores.append(scores.mean())

In [169]:
# # changing to misclassification error
# MSE = [1 - x for x in cv_scores]

# plt.figure()
# plt.figure(figsize=(15,10))
# plt.title('The optimal number of neighbors', fontsize=20, fontweight='bold')
# plt.xlabel('Number of Neighbors K', fontsize=15)
# plt.ylabel('Misclassification Error', fontsize=15)
# sns.set_style("whitegrid")
# plt.plot(k_list, MSE)

# plt.show()

In [170]:
# best_k = k_list[MSE.index(min(MSE))]
# print("The optimal number of neighbors is %d." % best_k)